# 4. Practice on news annotation
使用上一節課得到的 news_annotation 建立模型  
1. 將斷詞完的 cln_content 的欄位轉成 tf-idf
2. 用 polarity_score 來做 Linear Model
3. 用 sentiment 來做 Logistic Regression Model

## Preprocessing

In [1]:
import pandas as pd
from pathlib import Path
data_folder = Path("../data/")

In [2]:
news = pd.read_pickle(data_folder / 'news_annotation.p')

In [3]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 3 columns):
cln_content       381 non-null object
polarity_score    381 non-null int64
sentiment         381 non-null object
dtypes: int64(1), object(2)
memory usage: 9.0+ KB


In [4]:
news.head()

,cln_content,polarity_score,sentiment
0,國民黨 高雄 市長 候選人 韓國瑜 昨 接受 媒體 專訪 時稱 當選 市長 後 禁止 政治 ...,15,1
1,國民黨 高雄 市長 候選人 韓國瑜 今天 台北 大安區 舉辦 第二場 北漂 青年 座談 表示...,28,1
2,韓流北 漂 第二場 國民黨 高雄 市長 韓國瑜 主攻 北漂 族群 昨天 舉行 北 漂族 見面...,17,1
3,國民黨 高雄 市長 候選人 韓國瑜 日前 接受 專訪 時 指出 高雄 市長 所有 高雄 街頭...,12,1
4,國民黨 高雄 市長 候選人 韓國瑜 接受 中時 電子報 專訪 稱 當選 市長 後 未來 政治...,5,1


In [5]:
news.sentiment = news.sentiment.astype('category')

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
news_tfidf = v.fit_transform(news.cln_content)

In [7]:
news_tfidf.shape

(381, 16479)

## Linear Model

In [8]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    news_tfidf, 
    news.polarity_score,
    test_size=0.3, 
    random_state=7)

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

regr = LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

In [10]:
print('Coefficients: \n', regr.coef_)

print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Coefficients: 
 [3.63835662 0.45605763 1.81917831 ... 0.         0.34342381 0.04584367]
Mean squared error: 170.07
Variance score: 0.24


## Logistic Regression Model

In [11]:
import sklearn
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    news_tfidf, 
    news.sentiment,
    test_size=0.3, 
    random_state=0)

In [12]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, lr.predict(X_test))

0.9391304347826087

In [14]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lr, news_tfidf, news.sentiment, cv=5)
print(scores)

[0.93506494 0.93506494 0.93421053 0.93421053 0.94666667]


In [15]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.94 (+/- 0.01)
